In [ ]:
import pandas as pd
import polars as pl
from datetime import datetime, timedelta, date

semana = date.today().isocalendar().week

In [ ]:
clubs = (
    pl.read_csv(
        "https://raw.githubusercontent.com/IvoVillanueva/acb2026/refs/heads/main/logos_calendario/clubs2026.csv"
    )
    .with_columns(pl.col("equipo").str.replace("Baskonia", "Baskonia Kosner"))
    .with_columns(
        pl.when(pl.col("equipo") == "Baskonia Kosner")
        .then(pl.lit("#000000"))
        .otherwise(pl.col("color"))
        .alias("color"),
        pl.when(pl.col("equipo") == "Baskonia Kosner")
        .then(
            pl.lit(
                "https://static.acb.com/img/www/clubes2025/2526KosnerBaskoniaLogoNegativo.png"
            )
        )
        .otherwise(pl.col("logo"))
        .alias("logo"),
    )
)

clubs

equipo,logo,color,url,abb
str,str,str,str,str
"""Barça""","""https://static.acb.com/img/www…","""#154284""","""https://www.acb.com/club/plant…","""BAR"""
"""Baskonia Kosner""","""https://static.acb.com/img/www…","""#000000""","""https://www.acb.com/club/plant…","""BKN"""
"""Bàsquet Girona""","""https://static.acb.com/img/www…","""#B32836""","""https://www.acb.com/club/plant…","""GIR"""
"""BAXI Manresa""","""https://static.acb.com/img/31/…","""#D5001E""","""https://www.acb.com/club/plant…","""BAX"""
"""Casademont Zaragoza""","""https://static.acb.com/img/www…","""#C8102E""","""https://www.acb.com/club/plant…","""CAZ"""
…,…,…,…,…
"""Río Breogán""","""https://static.acb.com/logos/1…","""#85ADEA""","""https://www.acb.com/club/plant…","""BRE"""
"""Surne Bilbao Basket""","""https://static.acb.com/img/www…","""#000000""","""https://www.acb.com/club/plant…","""SBB"""
"""UCAM Murcia""","""https://static.acb.com/img/www…","""#A51A14""","""https://www.acb.com/club/plant…","""UCM"""


In [19]:
data_euroleague = (
    pl.read_csv(
        "https://raw.githubusercontent.com/IvoVillanueva/BOXSCORES-EUROLEAGE-2025_26/refs/heads/main/data/euroleague_boxscore_2025_26.csv"
    )
    .with_columns(
        pl.col("fecha").str.to_datetime("%Y-%m-%d %H:%M:%S").alias("fecha"),
        pl.col("minutes").str.replace("DNP", "0:00").alias("minutes"),
    )
    .with_columns(
        # --- Normalizar nombres de equipo ---
        pl.when(pl.col("team_name") == "KOSNER BASKONIA VITORIA-GASTEIZ")
        .then(pl.lit("Baskonia Kosner"))
        .when(pl.col("team_name") == "REAL MADRID")
        .then(pl.lit("Real Madrid"))
        .when(pl.col("team_name") == "FC BARCELONA")
        .then(pl.lit("Barça"))
        .when(pl.col("team_name") == "VALENCIA BASKET")
        .then(pl.lit("Valencia Basket"))
        .otherwise(pl.col("team_name"))
        .alias("team_name"),
        # --- firstName ---
        pl.when(pl.col("player").str.contains(","))
        .then(pl.col("player").str.split(", ").list.get(1).str.to_titlecase())
        .otherwise(pl.col("player").str.to_titlecase())
        .alias("firstName"),
        # --- lastName ---
        pl.when(pl.col("player").str.contains(","))
        .then(pl.col("player").str.split(", ").list.get(0).str.to_titlecase())
        .otherwise(pl.lit(None))
        .alias("lastName"),
        # --- nombre completo ---
        pl.when(pl.col("player").str.contains(","))
        .then(
            pl.col("player").str.split(", ").list.get(1).str.to_titlecase()
            + pl.lit(" ")
            + pl.col("player").str.split(", ").list.get(0).str.to_titlecase()
        )
        .otherwise(pl.col("player").str.to_titlecase())
        .alias("player"),
        # --- minutos redondeados ---
        (
            pl.col("minutes").str.split(":").list.get(0).cast(pl.Int64)
            + pl.when(pl.col("minutes").str.split(":").list.get(1).cast(pl.Int64) >= 30)
            .then(1)
            .otherwise(0)
        ).alias("minutes"),
    )
    .with_columns(
        # --- nombres correctos ---
        pl.when(pl.col("player") == "Xabi Lopez-Arostegui")
        .then(pl.lit("Xabier López-Arostegui"))
        .when(pl.col("player") == "Willy Hernangomez")
        .then(pl.lit("Willy Hernangómez"))
        .when(pl.col("player") == "Walter Tavares")
        .then(pl.lit("Edy Tavares"))
        .when(pl.col("player") == "Facundo Campazzo")
        .then(pl.lit("Facu Campazzo"))
        .when(pl.col("player") == "Nicolas Laprovittola")
        .then(pl.lit("Nico Laprovittola"))
        .when(pl.col("player") == "Andres Feliz")
        .then(pl.lit("Andrés Feliz"))
        .otherwise(pl.col("player"))
        .alias("player")
    )
    .filter(
        (
            pl.col("team_name").is_in(
                ["Baskonia Kosner", "Real Madrid", "Barça", "Valencia Basket"]
            )
        )
        & (pl.col("semana") == semana)
    )
    .select(
        [
            "competicion",
            "fecha",
            "semana",
            "ronda",
            pl.col("id_match").alias("partido"),
            "firstName",
            "lastName",
            "player",
            pl.col("team_name").alias("equipo"),
            pl.col("minutes").alias("min"),
            pl.col("points").alias("pts"),
            pl.col("total_rebounds").alias("reb"),
            pl.col("assistances").alias("ast"),
            pl.col("steals").alias("br"),
            pl.col("turnovers").alias("bp"),
            pl.col("blocks_favour").alias("tap"),
            pl.col("fouls_commited").alias("fa"),
            pl.col("valuation").alias("val"),
            pl.col("plusminus").alias("mm"),
        ]
    )
)

data_euroleague

competicion,fecha,semana,ronda,partido,firstName,lastName,player,equipo,min,pts,reb,ast,br,bp,tap,fa,val,mm
str,datetime[μs],i64,i64,i64,str,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""Euroleague""",2026-01-06 19:00:00,2,20,192,"""Brancou""","""Badio""","""Brancou Badio""","""Valencia Basket""",21,15,1,4,0,2,0,2,12,6
"""Euroleague""",2026-01-06 19:00:00,2,20,192,"""Kameron""","""Taylor""","""Kameron Taylor""","""Valencia Basket""",26,13,5,3,1,1,0,1,17,16
"""Euroleague""",2026-01-06 19:00:00,2,20,192,"""Nathan""","""Reuvers""","""Nathan Reuvers""","""Valencia Basket""",15,6,1,0,0,2,0,2,1,5
"""Euroleague""",2026-01-06 19:00:00,2,20,192,"""Jaime""","""Pradilla""","""Jaime Pradilla""","""Valencia Basket""",20,9,5,2,0,0,0,1,12,2
"""Euroleague""",2026-01-06 19:00:00,2,20,192,"""Sergio""","""De Larrea""","""Sergio De Larrea""","""Valencia Basket""",6,0,4,0,1,0,0,1,5,2
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Euroleague""",2026-01-08 19:30:00,2,21,203,"""Omari""","""Moore""","""Omari Moore""","""Valencia Basket""",17,3,3,3,1,1,0,0,4,1
"""Euroleague""",2026-01-08 19:30:00,2,21,203,"""Neal""","""Sako""","""Neal Sako""","""Valencia Basket""",10,2,1,1,0,0,0,0,4,-5
"""Euroleague""",2026-01-08 19:30:00,2,21,203,"""Darius""","""Thompson""","""Darius Thompson""","""Valencia Basket""",24,17,2,4,2,1,1,1,22,9


In [ ]:
df_partidos_euroleague = (
    data_euroleague.group_by(["player", "equipo"])
    .agg(
        [
            pl.col("partido").n_unique().alias("jug"),
            pl.col("min").mean(),
            pl.col("pts").mean(),
            pl.col("reb").mean(),
            pl.col("ast").mean(),
            pl.col("br").mean(),
            pl.col("bp").mean(),
            pl.col("tap").mean(),
            pl.col("fa").mean(),
            pl.col("mm").mean(),
            pl.col("val").mean(),
        ]
    )
    .sort("player")
    .join(clubs, on="equipo")
)
df_partidos_euroleague

player,equipo,jug,min,pts,reb,ast,br,bp,tap,fa,mm,val,logo,color,url,abb
str,str,u32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,str
"""Alberto Abalde""","""Real Madrid""",2,20.0,3.5,2.0,2.0,1.5,1.5,0.0,2.5,13.0,4.5,"""https://static.acb.com/img/www…","""#FFFFFF""","""https://www.acb.com/club/plant…","""RMB"""
"""Alex Len""","""Real Madrid""",2,5.0,1.0,0.0,0.5,0.0,0.0,0.5,0.5,-5.0,1.5,"""https://static.acb.com/img/www…","""#FFFFFF""","""https://www.acb.com/club/plant…","""RMB"""
"""Andres Feliz""","""Real Madrid""",2,18.5,8.0,1.0,2.5,0.5,1.0,0.0,1.0,8.5,8.5,"""https://static.acb.com/img/www…","""#FFFFFF""","""https://www.acb.com/club/plant…","""RMB"""
"""Brancou Badio""","""Valencia Basket""",2,20.0,11.5,1.5,2.5,0.0,1.5,0.0,2.0,1.0,9.0,"""https://static.acb.com/img/23/…","""#FC6C0F""","""https://www.acb.com/club/plant…","""VBC"""
"""Braxton Key""","""Valencia Basket""",2,18.5,8.0,3.0,2.5,1.0,1.0,1.0,3.5,8.0,10.5,"""https://static.acb.com/img/23/…","""#FC6C0F""","""https://www.acb.com/club/plant…","""VBC"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Usman Garuba""","""Real Madrid""",2,10.5,4.5,3.0,0.0,1.5,0.0,0.5,1.0,6.5,8.0,"""https://static.acb.com/img/www…","""#FFFFFF""","""https://www.acb.com/club/plant…","""RMB"""
"""Walter Tavares""","""Real Madrid""",2,24.0,10.5,10.0,0.0,0.0,1.0,0.5,2.5,10.0,22.5,"""https://static.acb.com/img/www…","""#FFFFFF""","""https://www.acb.com/club/plant…","""RMB"""
"""Willy Hernangomez""","""Barça""",1,22.0,10.0,10.0,1.0,1.0,3.0,3.0,4.0,6.0,22.0,"""https://static.acb.com/img/www…","""#154284""","""https://www.acb.com/club/plant…","""BAR"""
